### Model Deployment in 2024


### Hugginface face Space


https://www.youtube.com/watch?app=desktop&v=ZBXNyOPv6mM

https://www.youtube.com/watch?v=I9eK7idNCGw

### Streamlit Community Cloud

### deployment with mlflow,github actions

### Content
- Loan_id A unique loan number assigned to each loan customers

- Loan_status Whether a loan is paid off, in collection, new customer yet to payoff, or paid off after the collection efforts

- Principal Basic principal loan amount at the origination

- terms Can be weekly (7 days), biweekly, and monthly payoff schedule

- Effective_date When the loan got originated and took effects

- Due_date Since it’s one-time payoff schedule, each loan has one single due date

- Paidoff_time The actual time a customer pays off the loan

- Pastdue_days How many days a loan has been past due

- Age, education, gender A customer’s basic demographic information

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder,LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer

dir_loc = "C:/Users/nboateng/OneDrive - Nice Systems Ltd/Documents/Deployment_Dashboards/modeldeployment2024/"
#data location on kaggle https://www.kaggle.com/datasets/zhijinzhai/loandata
#df = pd.read_csv("C:/Users/nboateng/OneDrive - Nice Systems Ltd/Documents/Deployment_Dashboards/Model_Deployment_2024/Loan payments data.csv", parse_dates = ['effective_date', 'paid_off_time','due_date'])
df = pd.read_csv( dir_loc + "Loan payments data.csv")

df.head()

,Loan_ID,loan_status,Principal,terms,effective_date,due_date,paid_off_time,past_due_days,age,education,Gender
0,xqd20166231,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/14/2016 19:31,NaN,45,High School or Below,male
1,xqd20168902,PAIDOFF,1000,30,9/8/2016,10/7/2016,10/7/2016 9:00,NaN,50,Bechalor,female
2,xqd20160003,PAIDOFF,1000,30,9/8/2016,10/7/2016,9/25/2016 16:58,NaN,33,Bechalor,female
3,xqd20160004,PAIDOFF,1000,15,9/8/2016,9/22/2016,9/22/2016 20:00,NaN,27,college,male
4,xqd20160005,PAIDOFF,1000,30,9/9/2016,10/8/2016,9/23/2016 21:36,NaN,28,college,female


In [5]:
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer




def month_date(df):
    #d =  df.apply(lambda x:x.str.slice(5, 7)).astype(float)
    d =  df.apply(lambda x: pd.to_datetime(x).dt.month)
    return d


def quarter_date(df):
    d = df.apply(lambda x: pd.to_datetime(x).dt.quarter)
    return d


def cast_float(df):
    #d = df.apply(lambda x: pd.DataFrame(x).astype(float))
    return df.astype(float)

def to_object(x):
  return pd.DataFrame(x).astype(object)

def to_numeric(x):
  return pd.DataFrame(x).astype(float)

fun_tr = FunctionTransformer(to_object)

num_tr = FunctionTransformer(to_numeric)

get_month_date = FunctionTransformer(month_date)    
 
get_quarter_date = FunctionTransformer(quarter_date)

cast_col_float = FunctionTransformer(to_numeric)




ct = make_column_transformer(
    
     (get_month_date,['due_date','paid_off_time']),
    (get_quarter_date,['effective_date'])
)


cast_num = make_column_transformer(
    
     (cast_float,['Principal','terms','past_due_days'	,'age'])
   
)

import datetime as dt
from datetime import datetime


ct.fit_transform(df)

#cast_num.fit_transform(df)

#y = fun_tr.fit_transform(pd.DataFrame({'a':[1,2,3]}))
#y = num_tr.fit_transform(pd.DataFrame({'a':[1,2,3]}))
y = num_tr.fit_transform(df[['Principal','terms']])
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Principal  500 non-null    float64
 1   terms      500 non-null    float64
dtypes: float64(2)
memory usage: 7.9 KB


from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBClassifier,XGBRegressor


# Features and target variable
X = df.drop(columns=['Loan_ID',	'loan_status'])
y = df['loan_status']

# Columns to be scaled
numeric_features = ['Principal','terms','past_due_days'	,'age']

# Column to be binned and one-hot encoded
categorical_features = ['education',	'Gender']

# Create transformers
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='median')),
    ('imputer', SimpleImputer(fill_value= -9999, strategy='constant'))
    #('scaler', MinMaxScaler())
])
categorical_transformer = Pipeline(steps=[
    ('bin', KBinsDiscretizer(n_bins=6, encode='ordinal', strategy='quantile')),
    #('encoder', OneHotEncoder(handle_unknown='ignore'))
   ('encoder', OrdinalEncoder(handle_unknown= 'use_encoded_value', unknown_value=-1)),
])

# Combine all transformers into a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create and evaluate the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression())])
pipeline

In [9]:
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder,StandardScaler,OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from xgboost import XGBClassifier,XGBRegressor
from sklearn import *
from sklearn.metrics import roc_curve, roc_auc_score,balanced_accuracy_score,f1_score,confusion_matrix

# Columns to be scaled
numeric_features = ['Principal','terms','past_due_days'	,'age']

# Column to be binned and one-hot encoded
categorical_features = ['education',	'Gender']

date_cols =  ['due_date','paid_off_time','effective_date']



target_column  =  ['loan_status']

feat_list  =  numeric_features  +  categorical_features +   date_cols  



categorical_transformer = Pipeline(steps=[
    #('encoding',LabelEncoder()),
    ('ordinal', OrdinalEncoder(handle_unknown= 'use_encoded_value', unknown_value=-1)),
     #('label_encoding',MyLabelEncoder()),
    #('imputer', SimpleImputer(fill_value= -9999, strategy='constant'))
     ('imputer', SimpleImputer( strategy='median')),
    #('encoding',OrdinalEncoder(categories='auto'))
   

])


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(fill_value=-9999, strategy='constant', add_indicator=True)),
    #('scaler', StandardScaler()) 
     ('cast_float',cast_col_float)
    ])


date_transformer = Pipeline(steps=[
    ('date', get_quarter_date)
    ])




xgb_classifier = xgb.XGBClassifier(
             seed=1,
              n_jobs = -1,
              max_depth = 5,
              learning_rate =  0.1,
              min_child_weight= 2, 
              #min_samples_split= 0.9,
              n_estimators= 50,
              #eta = 0.1, 
              verbose = 1, 
              gamma=0.05,
              #nrounds = 100
              objective='multi:softmax', 
              num_class=3,
              eval_metric =  metrics.auc,              #metrics.r2_score,     mean_absolute_error  #"aucpr",    # "aucpr",  #aucpr, auc
              subsample = 0.7,
              colsample_bytree =0.8,
              max_delta_step=1,
              verbosity=1,
              tree_method='hist')

# Combine all transformers into a preprocessor using ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_transform', numeric_transformer, numeric_features),
        ('date_eng_feat',date_transformer,date_cols),
        ('categorical_encoding', categorical_transformer, categorical_features),
        #('label_encoder', label_transformer, target_column)
       
        ])


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      #('dropfeature',UniqueDropper()),
                      #('anova', SelectPercentile(chi2)),
                      # ('feature_selection', SelectFromModel(LinearSVC(penalty="l1"))),
                      ('classifier',xgb_classifier )])


# Define the label encoder
label_encoder = LabelEncoder()


from sklearn.model_selection import train_test_split

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(df[feat_list], df[target_column], test_size=0.3, random_state=42)


pipeline.fit(X=X_train, y= label_encoder.fit_transform(y_train.values.ravel()))


c:\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [17:32:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numeric_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(add_indicator=True,
                                                                                 fill_value=-9999,
                                                                                 strategy='constant')),
                                                                  ('cast_float',
                                                                   FunctionTransformer(func=<function to_numeric at 0x000001DC372C5120>))]),
                                                  ['Principal', 'terms',
                                                   'past_due_days', 'age']),
                                                 ('date_eng_feat',
                                                  Pipeline(steps=[('da...
                               feature_types=None, gamma=0.05, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=1,
                               max_depth=5, max_leaves=None, min_child_weight=2,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=50, n_jobs=-1,
                               num_class=3, num_parallel_tree=None, ...))])

In [11]:
# print train and test set shape
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

# generate predictions
y_pred = pipeline.predict(X_test)
#y_pred_prob = pipeline.predict_proba(X_test)[:, 1]
y_pred_prob = pipeline.predict_proba(X_test)
y_pred_label = pipeline.predict(X_test)
y_test_label = label_encoder.fit_transform(y_test.values.ravel())


auc = roc_auc_score(y_test_label,y_pred_prob,multi_class='ovr')
print('auc :{}'.format(auc))

balanced_accuracy = balanced_accuracy_score(y_test_label,y_pred_label)
print('balanced_accuracy  :{}'.format(balanced_accuracy ))
#f1 = f1_score(y_test_label,y_pred_label,'micro')
f1 = f1_score(y_test_label, y_pred_label, average='macro')
print('f1  :{}'.format(f1 ))


import joblib

joblib.dump(pipeline, dir_loc + 'pipeline.pkl')
y_pred_label


X_train: (350, 9)
X_test: (150, 9)
y_train: (350, 1)
y_test: (150, 1)
auc :1.0
balanced_accuracy  :1.0
f1  :1.0


array([0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1,
       2, 0, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2,
       1, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 1, 1, 2, 2, 1, 2, 0, 2, 2, 1,
       0, 2, 0, 1, 0, 0, 1, 0, 2, 2, 1, 2, 2, 1, 2, 0, 0, 2, 0, 2, 0, 2,
       2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1])

In [12]:
{'COLLECTION':0,
  'PAIDOFF':2,
  'COLLECTION_PAIDOFF':1}

# Reverse the encoding
original_labels = label_encoder.inverse_transform(y_test_label)
print(original_labels)

['COLLECTION' 'PAIDOFF' 'COLLECTION' 'PAIDOFF' 'PAIDOFF' 'COLLECTION'
 'COLLECTION' 'PAIDOFF' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'COLLECTION_PAIDOFF' 'PAIDOFF' 'COLLECTION' 'COLLECTION'
 'COLLECTION_PAIDOFF' 'PAIDOFF' 'COLLECTION' 'COLLECTION_PAIDOFF'
 'COLLECTION_PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'COLLECTION'
 'PAIDOFF' 'COLLECTION_PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'COLLECTION' 'COLLECTION_PAIDOFF' 'COLLECTION' 'COLLECTION_PAIDOFF'
 'PAIDOFF' 'PAIDOFF' 'COLLECTION' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'PAIDOFF' 'COLLECTION' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'PAIDOFF' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'PAIDOFF'
 'PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF' 'COLLECTION'
 'COLLECTION_PAIDOFF' 'PAIDOFF' 'PAIDOFF' 'COLLECTION_PAIDOFF' 'PAIDOFF'
 'PAIDOFF' 'COLLECTION' 'COLLECTION' 'COLLECTION' 'PAIDOFF' 'PAIDOFF'
 'COLLECTIO

In [13]:
y_test_label

array([0, 2, 0, 2, 2, 0, 0, 2, 2, 1, 2, 2, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1,
       2, 0, 2, 1, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2, 2, 0, 2, 1, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 1, 2, 2,
       1, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2, 1, 1, 2, 2, 1, 2, 0, 2, 2, 1,
       0, 2, 0, 1, 0, 0, 1, 0, 2, 2, 1, 2, 2, 1, 2, 0, 0, 2, 0, 2, 0, 2,
       2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1])

In [ ]:
#pip install --upgrade modelbit --user

  Using cached modelbit-0.42.1-py3-none-any.whl.metadata (2.1 kB)
Using cached modelbit-0.42.1-py3-none-any.whl (150 kB)
  Attempting uninstall: modelbit
    Found existing installation: modelbit 0.42.0
    Uninstalling modelbit-0.42.0:
      Successfully uninstalled modelbit-0.42.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:

# install modelbit
#!pip install modelbit

# run on top of your notebook
import modelbit
mb = modelbit.login()

### Creating an Inference function
The first step is to create a Python function for inference which uses the predict or predict_proba method from sklearn.

In [42]:
import pandas as pd
import numpy as np

# first define function
def predict_loan_default(Principal=0, terms=0,past_due_days=0,
                        age=0,education = 'High School' ,Gender = 'male',
                        due_date= '9/25/2016',paid_off_time='9/25/2016' ,effective_date = '9/25/2016' ):
#def predict_loan_default(Principal:float, terms:float,past_due_days:float,age:float,education:str ,Gender:str,due_date:str ,paid_off_time:str ,effective_date:str)-> float:
   
  """
  Predict the probability of loan default using a pre-trained machine learning pipeline.
 
  Args:
     
      Principal (int): loan principal
      terms (int):
      past_due_days (int):	
      age (int):
      education : str	
      Gender :str
      due_date :str
      paid_off_time :str
      effective_date :str

  Returns:
      float: Probability of loan default.
  """
  data = pd.DataFrame([[Principal, terms,past_due_days,
                        age,education ,Gender,
                        due_date,paid_off_time ,effective_date ]],
                                             columns = [ 'Principal', 'terms','past_due_days','age','education' ,'Gender','due_date' ,'paid_off_time' ,'effective_date'])
  
  for col in [ 'Principal', 'terms','past_due_days','age']:
    data[col] = data[col].astype(float)
  pred_prob = pipeline.predict_proba(data) 
  

  #pred_label = pipeline.predict(pd.DataFrame([[Principal, terms,past_due_days,age,education ,Gender,due_date ,paid_off_time ,effective_date ]],
  #                                           columns = [ 'Principal', 'terms','past_due_days','education' ,'Gender','due_date' ,'paid_off_time' ,'effective_date'])) 
  
  #return data
  return  dict(zip(['COLLECTION','PAIDOFF','COLLECTION_PAIDOFF'],pred_prob[0]))                            

In [44]:
predict_loan_default(800, 15,74,29,'High School or Below','male','9/25/2016','9/12/2016' ,'9/11/2016' )
#print(len([800, 15,74,29,'High School or Below','male','9/25/2016','9/12/2016' ,'9/11/2016' ]))
#print(len([ 'Principal', 'terms','past_due_days','age','education' ,'Gender','due_date' ,'paid_off_time' ,'effective_date']))
#print(len([Principal, terms,past_due_days,age,education ,Gender,due_date ,paid_off_time ,effective_date ]))

{'COLLECTION': 0.052470516,
 'PAIDOFF': 0.91186875,
 'COLLECTION_PAIDOFF': 0.03566072}

In [33]:
d= pd.DataFrame([[800, 15,74,29,'High School or Below','male','9/25/2016','9/12/2016' ,'9/11/2016' ]],
                                             columns = [ 'Principal', 'terms','past_due_days','age','education' ,'Gender','due_date' ,'paid_off_time' ,'effective_date'])

d['past_due_days']  = d['past_due_days'].astype(float)

print(pipeline.predict_proba(d).argmax() )
print(pipeline.predict_proba(d)[0] )

#X_test.info()
#pipeline.predict_proba()

#pipeline.predict_proba(X_test)
#print(d)
#X_test.head()
d.info()

#dict(zip(['COLLECTION','PAIDOFF','COLLECTION_PAIDOFF'],pipeline.predict_proba(d)[0]))

#predict_loan_default([800, 15,74,29,'High School or Below','male','9/25/2016','9/12/2016' ,'9/11/2016' ])

1
[0.05247052 0.91186875 0.03566072]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Principal       1 non-null      int64  
 1   terms           1 non-null      int64  
 2   past_due_days   1 non-null      float64
 3   age             1 non-null      int64  
 4   education       1 non-null      object 
 5   Gender          1 non-null      object 
 6   due_date        1 non-null      object 
 7   paid_off_time   1 non-null      object 
 8   effective_date  1 non-null      object 
dtypes: float64(1), int64(3), object(5)
memory usage: 204.0+ bytes


In [251]:
#!pip install fast_ml --upgrade
#predict_loan_default(29, 800, 15,74,'High School or Below','male','9/25/2016','' ,'9/11/2016')
X_test.head()

,Principal,terms,past_due_days,age,education,Gender,due_date,paid_off_time,effective_date
361,800,15,74.0,29,High School or Below,male,9/25/2016,NaN,9/11/2016
73,1000,15,NaN,34,college,male,9/25/2016,9/25/2016 9:00,9/11/2016
374,1000,30,59.0,29,college,male,10/10/2016,NaN,9/11/2016
155,1000,30,NaN,36,High School or Below,female,10/11/2016,10/11/2016 13:00,9/12/2016
104,1000,15,NaN,35,college,male,9/25/2016,9/22/2016 13:15,9/11/2016


### Deploy Machine Learning Pipeline on the cloud using Docker Container

### Build Web Application
This tutorial is not focused on building a Flask application. It is only discussed here for completeness. Now that our machine learning pipeline is ready we need a web application that can connect to our trained pipeline to generate predictions on new data points in real-time. We have created the web application using Flask framework in Python. There are two parts of this application:

- Front-end (designed using HTML)
- Back-end (developed using Flask)